[19 卷积层【动手学深度学习v2】](https://www.bilibili.com/video/BV1L64y1m7Nh?spm_id_from=333.999.0.0)

### 重新考察全连接层
- 将输入输出变为举证(宽度,高度)也就是接下来的(k,L)
- 将权重变形为4-D张量(h,w)-(h',w') 也就是输入增加了宽度和高度两个矩阵
- 为什么是4维不是三维，因为单个像素可能包含有其他信息，因此最好是用二维矩阵表示
- $$h_{i,j} = \sum_{k,l} W_{i,j,k,l} x_{k,l} = \sum_{a,b} V_{i,j,a,b}x_{i+a,j+b}$$
- V是W的重新索引$V_{i,j,a,b} = W_{i,j,i+a,j+b}$
- 相当于做转换：
- $k=i+a $
- $l=j+b$

### 原则1-平移不变性
- x的平移导致h的平移 $h_{i,j} = \sum_{a,b} v_{i,j,a,b}x_{i+a,j+b}$
- v不应依赖与(i,j)
- 解决方案：$V_{i,j,a,b} = V_{a,b}$ 
- 注意：这里的不依赖是假设i,j和v无关，实际操作中,可以理解为$i,j$对w也就是用于卷积的小矩阵的系数没有影响
也就是说从头到尾采用固定的卷积核（小矩阵）去依次乘积
- 那么进一步简化为$h_{i,j} = \sum_{a,b} v_{a,b}x_{i+a,j+b}$
- 这就是2维卷积（交叉相关）

### 原则2-局部性
- $h_{i,j} = \sum_{a,b} v_{a,b}x_{i+a,j+b}$
- 当评估$h_{i,j}$的时候，我们不应该用远离$x_{i,j}$的参数
- 解决方案：当$|a|,|b|>\Delta$时候，使得$v_{a,b}=0$
- $$h_{i,j} = \sum_{a=-\Delta}^\Delta \sum_{b=-\Delta}^\Delta v_{a,b}x_{i+a,j+b}$$
- 这里的意思是，卷积只对$±\Delta$范围内的数据进行卷积

可以看到卷积最后相当于形成了一个特殊的全连接层

### 二维卷积层
- 输入$X:n_h * n_w$
- 核$W: k_h * k_w$
- 偏差$b \in R$ 
- 输出 $Y:(n_h - k_n +1) \times(n_w - k_w +1)$
  $$Y = X * W +b$$
- W和b都是可以学习的参数

<img src="picture\屏幕截图 2022-04-19 012840.png"></a>

- 上图展示了不同类似的卷积核能够起到的作用

### 交叉相关与卷积
- 二维交叉相关
- $$y_{i,j} = \sum_{a=1}^h \sum_{b=1}^w w_{a,b}x_{i+a,j+b}$$
- 二维卷积
- $$y_{i,j} = \sum_{a=1}^h \sum_{b=1}^w w_{-a,-b}x_{i+a,j+b}$$
- 由于对称性，实际使用中没有区别

<img src="picture\屏幕截图 2022-04-19 014213.png"></a>

### 总结
- 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
- 核矩阵和偏移是可学习的参数
- 核矩阵的大小是超参数

### 代码实现卷积层

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X,K):
    """计算二维互相关运算"""
    h,w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
# 实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        # kernel size为超参数，也就是核函数的大小
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
#  图像中⽬标的边缘检测
# 如下是卷积层的⼀个简单应⽤：通过找到像素变化的位置，来检测图像中不同颜⾊的边缘。⾸先，我们构造
# ⼀个6 X 8 像素的⿊⽩图像。中间四列为⿊⾊（0），其余像素为⽩⾊（1）。
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
# 神奇的小核
K = torch.tensor([[1.0, -1.0]])

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
# 现在我们将输⼊的⼆维图像转置，再进⾏如上的互相关运算。其输出如下，之前检测到的垂直边缘消失了。
# 不出所料，这个卷积核K只可以检测垂直边缘，⽆法检测⽔平边缘。
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### 学习卷积核
如果我们只需寻找⿊⽩边缘，那么以上[1, -1] 的边缘检测器⾜以。然而，当有了更复杂数值的卷积核，或
者连续的卷积层时，我们不可能⼿动设计过滤器。那么我们是否可以学习由X⽣成Y的卷积核呢？
现在让我们看看是否可以通过仅查看“输⼊-输出”对来学习由X ⽣成Y 的卷积核。我们先构造⼀个卷积层，
并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们⽐较Y 与卷积层输出的平⽅误差，然后计算
梯度来更新卷积核。为了简单起⻅，我们在此使⽤内置的⼆维卷积层，并忽略偏置。

In [9]:
# 构造⼀个⼆维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)
# 这个⼆维卷积层使⽤四维输⼊和输出格式（批量⼤⼩、通道、⾼度、宽度），
# 其中批量⼤⼩和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(10):
    # 卷积计算
    Y_hat = conv2d(X)
    # 均方误差
    l = (Y_hat - Y) ** 2
    # 梯度设为0
    conv2d.zero_grad()
    # 梯度下降
    l.sum().backward()
# 迭代卷积核
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 9.162
batch 4, loss 2.752
batch 6, loss 0.959
batch 8, loss 0.365
batch 10, loss 0.145
